In [1]:
%load_ext autoreload
%autoreload 2
from functools import partial
import os
import keras

from datasets import example_datasets, to_numpy, tensorflow_dataset, train_val_test_split
from models import mixture_poissons,poisson_glm
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, get_unperturbed_bpr_loss_func
from experiments import training_loop
from plotting_funcs import plot_losses, plot_frontier, plot_component_histograms, plot_winners
from bpr_trials import calc_bpr_many_trials

from distributions import ZeroInflatedDist, QuantizedNormal

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from glob import glob
import scipy

import matplotlib.pyplot as plt

2024-05-07 11:34:30.712317: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 11:34:30.712380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 11:34:30.725184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 11:34:30.763872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 11:34:46.929037: W tensorflow/compiler/tf2

In [2]:
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 15000
# K for bpr
K=3
seed=12343
train_pct = 1/3

In [3]:
consistent_4 = [QuantizedNormal(7, 0.1) for _ in range(4)]

highvar_4 = [ZeroInflatedDist(QuantizedNormal(10, 0.1), 1-0.7) for _ in range(4)]

powerball_4 = [ZeroInflatedDist(QuantizedNormal(100, 0.1), 0.9) for _ in range(4)]
dist_S = consistent_4 + highvar_4 +powerball_4


In [13]:
num_tracts_per_type = 4

probability_of_10 = 0.71
probability_of_100 = 0.1

outcomes = []
probabilities = []
num_10s = []
num_100s = []
bpr_denominators = []

for hit_10s in range(num_tracts_per_type+1):
    probability_of_10s = (probability_of_10**hit_10s*(1-probability_of_10)**(num_tracts_per_type-hit_10s))*scipy.special.comb(num_tracts_per_type, hit_10s)

    for hit_100s in range(num_tracts_per_type+1):

        bpr_denominator = min(K,hit_100s)*100
        if K - hit_100s > 0:
            bpr_denominator += min(hit_10s, K-hit_100s)*10
        if K -hit_100s-hit_10s>0:
            bpr_denominator += (K-hit_100s-hit_10s)*7 

        probability_of_100s = (probability_of_100**hit_100s*(1-probability_of_100)**(num_tracts_per_type-hit_100s))*scipy.special.comb(num_tracts_per_type, hit_100s)
        probabilities.append(probability_of_10s * probability_of_100s)
        bpr_denominators.append(bpr_denominator)

        num_10s.append(hit_10s)
        num_100s.append(hit_100s)



In [14]:
pick_7_bprs = []
pick_10_bprs = []
pick_100_bprs = []

for probability, denominator, hit_10s, hit_100s in zip(probabilities, bpr_denominators, num_10s, num_100s):
    pick_7_bprs.append(K*7/denominator*probability)
    pick_10_bprs.append(10*hit_10s/denominator*probability)
    pick_100_bprs.append(100*hit_100s/denominator*probability)


print(f'E[%BPR] for pick 7: {np.sum(pick_7_bprs)*100:.2f}')
print(f'E[%BPR] for pick 10: {np.sum(pick_10_bprs)*100:.2f}')
print(f'E[%BPR] for pick 100: {np.sum(pick_100_bprs)*100:.2f}')

E[%BPR] for pick 7: 53.79
E[%BPR] for pick 10: 71.32
E[%BPR] for pick 100: 29.35


In [16]:
# generate all possible length 8 binary vectors:
all_possible = []
for i in range(2**8):
    all_possible.append([int(x) for x in bin(i)[2:].zfill(8)])

start_idx_10 = 0
start_idx_100 = 4
num_tracts_per_type = 4

K=3
pick_10s_indicies =np.array([1,1,1,0,0,0,0,0]) 
pick_100s_indicies =np.array([0,0,0,0,1,1,1,0] )

probabilities = []
pick_7_bprs = []
pick_10_bprs = []
pick_100_bprs = []

for outcome in all_possible:
    

    hit_10s = np.sum(outcome[start_idx_10:start_idx_10+num_tracts_per_type])
    picked_10s = np.sum(outcome*pick_10s_indicies)
    hit_100s = np.sum(outcome[start_idx_100:start_idx_100+num_tracts_per_type])
    picked_100s = np.sum(outcome*pick_100s_indicies)

    probability_10s = (probability_of_10**hit_10s*(1-probability_of_10)**(num_tracts_per_type-hit_10s))
    probability_of_100s = (probability_of_100**hit_100s*(1-probability_of_100)**(num_tracts_per_type-hit_100s))
    probability = probability_10s*probability_of_100s
    probabilities.append(probability)

    bpr_denominator = min(K,hit_100s)*100
    if K - hit_100s > 0:
        bpr_denominator += min(hit_10s, K-hit_100s)*10
    if K -hit_100s-hit_10s>0:
        bpr_denominator += (K-hit_100s-hit_10s)*7 

    pick_7_bprs.append(K*7/bpr_denominator)
    pick_10_bprs.append(10*picked_10s/bpr_denominator)
    pick_100_bprs.append(100*picked_100s/bpr_denominator)

print(f'E[%BPR] for pick 7: {np.sum(pick_7_bprs*np.array(probabilities))*100:.2f}')
print(f'E[%BPR] for pick 10: {np.sum(pick_10_bprs*np.array(probabilities))*100:.2f}')
print(f'E[%BPR] for pick 100: {np.sum(pick_100_bprs*np.array(probabilities))*100:.2f}')

E[%BPR] for pick 7: 53.79
E[%BPR] for pick 10: 53.49
E[%BPR] for pick 100: 22.02
